<a href="https://colab.research.google.com/github/acse-xm421/ACSE-19-20/blob/master/acse-xm421_coursework_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1dFgNX9iQUfmBOdmUN2-H8rPxL3SLXmxn" width="400"/>

### ***Name***: Xuefei Mi
### ***CID***: 02084183

# Instructions:

Follow the instructions below to complete the coursework and submit it:

<br>

1. Complete your coursework using this provided Jupyter Notebook template (use Google Colab or your local machine. Your copy of the notebook should be named: `yourusername_coursework_II.ipynb`. And don't forget to fill in the two fields at the top of this notebook with your name and CID.

<br>

2. Once you have completed your answers, upload your final notebook to the repo you got from the github classroom link. Make sure to have all the answers in there:

   - **All the cells in your final Jupyter Notebook should be executed before saving and uploading to github in order to have the output of the cells available in the uploaded version** (images you plot, outputs of calculations, etc). We will not rerun code blocks in the notebooks, it is your responsibility to run them before uploading the notebook.

   - Add comments in the code to explain what you are doing at every step in the coding cells.

   - All answers requiring written answers (ie, not code) should be in markdown blocks in the Jupyter Notebook. This provided Jupyter Notebook template has allocated blocks for the questions, but **you can add any coding or markdown blocks you need**.

<br>

3. The coursework is released on **Friday 16 December at 14:00h UK time**, and the answers have to be submitted by **Friday 16 December, 17:00h UK time**. We will not accept late submissions.

4. If you have questions during the coursework, come to me directly and I will address them. Usually, they will be relevant to the rest of the students and if this is the case, I will answer them for everybody. **DO NOT ask questions in the chat as this may give answers away to other students, if you prefer to, send the questions via chat to me privately, but do not use any of the general chats please.**

<br>

<br>

### The coursework consists of 8 questions you have to complete. You will find them below. 

- This is an open-book assessment, so feel free to browse the materials we have seen in the module to assist you with your answers.

<br>

---

<br>

In [1]:
## provided imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CelebA
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

device='cpu'

## **Question 1** [10 points]

Suppose we have a feed-forward network (FFN) designed to predict which class an image belongs to. Given the following architecture and input characteristics:

- Images are single channel and have a size of 100x105.
- The FFN has 4 hidden layers with 1001, 2002, 1345, and 1002 output nodes (output size of each layer).
- The FFN output size is 10 nodes.
- There are bias terms everywhere.
- The activation function in all layers except the output layer are ReLU.

Answer the following questions:
#### **1.1** Implement this network as an nn.Module.

In [8]:
class FFNq1(nn.Module):
    def __init__(self):
      super(FFNq1, self).__init__()
      self.fc1 = nn.Linear(100*105, 1001)
      self.fc2 = nn.Linear(1001, 2002)
      self.fc3 = nn.Linear(2002, 1345)
      self.fc4 = nn.Linear(1345, 1002)
      self.output = nn.Linear(1002, 10)
      self.act = nn.ReLU()
      self.actout = nn.Softmax()
        ### your code here

    def forward(self, x):
      x = torch.flatten(x, start_dim = 1)

      x = self.act(self.fc1(x))
      x = self.act(self.fc2(x))
      x = self.act(self.fc3(x))
      x = self.act(self.fc4(x))
      x = self.actout(self.output(x))

      return x
      
x= torch.rand(1, 1, 100, 105)
model = FFNq1()
output = model(x)


        ### your code here

<ipython-input-8-17a4f347213b>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.actout(self.output(x))


your answers here

#### **1.2** How many trainable parameters does the network have? Use a code block to calculate it and explain the process (where you get all the numbers you multiply and add to find the answer) in comments or in a markdown block.

Calculate the solution explicitly. If you want to use `torchsummary`, that's ok, **but only use it to validate that your answer is correct**. You will be assessed on the explicit calculation only.

In [9]:
sum(p.numel() for p in model.parameters())

16570262

In [10]:
# set_seed(42)

# Check number of parameters
nparam_layer1 = ((1*100*105+1) * (1001)) 
nparam_layer2 = ((1001+1) * 2002)
nparam_layer3 = ((2002+1) * 1345)
nparam_layer4 = ((1345+1) * 1002)
nparam_layer5 = ((1002+1) * 10)

print(nparam_layer1 + nparam_layer2 + nparam_layer3+nparam_layer4+nparam_layer5)

16570262


your answers here

#### **1.3** How would you compute the loss? Get an instance of the loss you would use (call it `criterion`) and explain what are the operations that this loss performs on the outputs.



In [ ]:
criterion = nn.CrossEntropy()


The cross-entropy is a loss function that can be used to quantify the difference between two probability distributions.[1] The output is the probability(p) of different classes, and we compare it with our true y, eg(1,0,0,0,0,0,0,0,0,0)

Loss_i = -y_i*log(p_i)
Loss = (1/N) * sum(Loss_i)

#### **1.4** How do you make predictions once the network is trained?

To clarify the question: how do you decide the class of a new image based on the output of the network?

Pick the class with the biggest probability.

\\

---

\\

## **Question 2** [10 points]

Suppose we have a convolutional neural network (CNN). Given the following architecture and input characteristics:

- Images have three channels, each with size 48x48
- The `nn.Module` of the network is:

In [11]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.c1 = nn.Conv2d(3, 12, kernel_size=3, stride=1, padding=1) 
    self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)               
    self.c3 = nn.Conv2d(12, 24, kernel_size=3, stride=1, padding=1)           
    self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)               
    self.c5 = nn.Conv2d(24, 1, kernel_size=3, stride=1, padding=1)                           
    self.act = nn.ReLU()                                          
    
  def forward(self, x):
    x = self.act(self.c1(x))                                      
    x = self.act(self.s2(x))                                      
    x = self.act(self.c3(x))                                     
    x = self.act(self.s4(x))                                     
    x = self.c5(x)
    return x             


                         

Answer the following questions:

#### **2.1** How many trainable parameters does this network have?

Calculate the solution explicitly. If you want to use `torchsummary`, that's ok, **but only use it to validate that your answer is correct**. You will be assessed on the explicit calculation only.

In [13]:
x= torch.rand(1, 3, 48, 48)
model = CNN()
output = model(x)

sum(p.numel() for p in model.parameters())


3169

In [20]:
nparam_layer1 = (3*3*3+1)*12
nparam_layer2 = (3*3*12+1)*24
nparam_layer3 = (3*3*24+1)*1

print(nparam_layer1 + nparam_layer2 + nparam_layer3)

3169


In [17]:
from torchsummary import summary
summ = summary(model, torch.Size((3, 48,48)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 48, 48]             336
              ReLU-2           [-1, 12, 48, 48]               0
         MaxPool2d-3           [-1, 12, 24, 24]               0
              ReLU-4           [-1, 12, 24, 24]               0
            Conv2d-5           [-1, 24, 24, 24]           2,616
              ReLU-6           [-1, 24, 24, 24]               0
         MaxPool2d-7           [-1, 24, 12, 12]               0
              ReLU-8           [-1, 24, 12, 12]               0
            Conv2d-9            [-1, 1, 12, 12]             217
Total params: 3,169
Trainable params: 3,169
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.79
Params size (MB): 0.01
Estimated Total Size (MB): 0.83
-----------------------------------------------

your answers here

#### **2.2** Does the number of trainable parameters you have calculated depends on the size of the input?

Provide a short explanation of your answer (one or two sentences)

No. Because the filter size only depends on the number of filters used in the convolutional layers and the size of the filters. Plus there is no FC layer, so it has nothing to do with input size.

\\

---

\\

## **Question 3** [10 points]

Given a dataset with 10000 samples, answer the following questions:

#### **3.1** The dataset is split in three: training dataset (80%), validation dataset (10%), and test dataset (10%). Which one of these statements is incorrect? Explain in one sentence why it is incorrect (don't need to explain the two that are correct).

- a) The training dataset is used to update the model parameters, but not the hyperparameters.
- b) The validation dataset is used to optimise hyperparameters, and after this is done, we can combine it with the training set for a final training with hyperparameters fixed.
- c) The test dataset that will be used to assess generalisation can also be used to optimise hyperparameters, but not for training weights and biases.







Answers (select one option in each case and explain the one that is False):
- a) True
- b) True
- c) False. The testset can only be used for assessing generalisation, or, it will happen data leak, resulting in inaccurate test score.

#### **3.2** How many iterations (model updates) will I have done in total if I train the network using the training dataset for 200 epochs?

- i) Full-batch gradient descent
- ii) Mini-batch gradient descent with a batch size of 99
- iii) Pure stochastic gradient descent (batch size of 1)

In [21]:
### Answers
total_num_iterations_i = 200
total_num_iterations_ii = (int(8000/99)+1)*200 = 16200
total_num_iterations_iii = 8000*200 = 1600000

#### **3.3** If the dataset is composed of images with three channels and 50x50 dimensions per channel and they are used as the input of a CNN where the first layer produces 25 feature maps of size 50x50 each. What would be the effect on the feature maps after applying a `nn.Dropout(0.2)` layer? And if we use `nn.Dropout2d(0.2)` layer instead?






In [ ]:
### your code (if you need it) here

For nn.Dropout(0.2), 20% of elements in the feature map will be dropout.

For nn.Dropout2d, it will drop out entire channels or spatial regions of the feature maps instead of each elements. So 5 feature map will be dropout.

\\

---

\\

## **Question 4** [15 points]

#### Given the network we used in class to illustrate how backpropagation works:

<br>

<center><img src="https://drive.google.com/uc?id=1YT8y2gqkDsrwbNyi0Lmo3YSHh3kcrf6H" width="600"/></center>

[link to the figure](https://drive.google.com/file/d/1YT8y2gqkDsrwbNyi0Lmo3YSHh3kcrf6H/view?usp=share_link) in case you cannot see it in the notebook.

<br>

#### Continue the derivation we saw in class to calculate the expression for the gradient of $w_1$:

<br>

$$\frac{\partial C}{\partial w_1}$$
 
#### where

$$C = \frac{1}{2}(a_4-y)^2$$

<br>

#### and assume that the activation function $g$ is now a sigmoid function $\sigma$. Choose what is the correct expression for the gradient of $w_1$ from the list below:

<br>

- a) $(a_4 -y)\;\sigma(z_4)\;(1-\sigma(z_4))\;w_3\; \sigma(z_3)\;(1-\sigma(z_3))\; w_2 \;\sigma(z_2)\;(1-\sigma(z_2))\; a_1$

- b) $(a_4 -y)\;\sigma(z_4)\;(1-\sigma(z_4))\;w_3\; \sigma(z_3)\;(1-\sigma(z_3))\; a_2 \;\sigma(z_2)\;(1-\sigma(z_2))\; a_1$

- c) $(a_4 -y)\;\sigma(z_4)\;w_3\; \sigma(z_3)\; w_2 \;\sigma(z_2)\; a_1$

- d) $(a_4 -y)\;\sigma(z_4)\;w_3\; \sigma(z_3)\; w_2 \;\sigma(z_2)\; w_1$

- e) *none of the above*

your answer here (pick an option and explain why it is the correct one). You don't have to add any mathematical derivation.

a. Assume there no z1. because $a2 = w1*a1+b1, z2 = \sigma(a2), a3 = w2*z2+b2$, and so on. So when using chain rule, we need to pass through several a and z. eg, The derivative of $$\frac{\partial a_3}{\partial z_2} = w_2$$, which is not associated with a2.
And also, we nned to calculate $$\frac{\partial z_2}{\partial a_2} = \sigma(a2)*(1-\sigma(a2))$$, which is missing in c and d options. 
So I choose a.

\\

---

\\

## **Question 5** [10 points]

Identify and briefly explain the errors in the following code snippets:

#### **5.1** The following network implementation is designed to take 1-channel two-dimensional inputs (1-chanel images) and classify them into 6 different classes. The code below has 4 errors. Find them and correct them so that you can run your network with a dummy input (code provided at the end of the cell). Indicate with comments in the code where you have made your changes:

***\[ you don't need to add any additional blocks in this question (5.1), only use the code block to make your changes and add comments on it\]***

In [26]:
class NetworkQ5(nn.Module):
  def __init__(self):
    super(NetworkQ5, self).__init__()

    self.c1 = nn.Conv2d(1, 12, kernel_size=5, stride=1, padding=2) #p4: it is a image, so it should be 2d. the channel cannot be taken into dimension.
    self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)               
    self.s2_dr = nn.Dropout2d(p=0.2)
    self.c3 = nn.Conv2d(12, 32, kernel_size=5, stride=1)           
    self.c3_bn = nn.BatchNorm2d(32) # p1:the batchnorm is following c3, so its num_features should be 32.
    self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)               
    self.c5 = nn.Linear(800, 120)   # p5: the flattened output of the last layer should be [1, 800], I have corrected it.                          
    self.f6 = nn.Linear(120, 84)  
    self.output = nn.Linear(84, 5)  # p2:we need to classify them into 6 classes, so the out_features in the final layer should be 6.
    self.act = nn.ReLU()                         
    self.actout = nn.Softmax()  # p3:when do classification, we need to use softmax() to convert our output to the probability of different classes.               
    
  def forward(self, x):
    x = self.act(self.c1(x))                                      
    x = self.act(self.s2_dr(self.s2(x)))                     
    x = self.act(self.c3(x))
    x = self.c3_bn(x)
    x = self.act(self.s4(x))                      
    print('before flatten',x.shape)               
    x = x.view(-1, x.size(1)*x.size(2)*x.size(3))               
    print('after',x.shape) 
    x = self.act(self.c5(x))                                    
    x = self.act(self.f6(x))                                    
    return self.actout(self.output(x))   #  the same as p3                                  
  
x = torch.randn((1, 1, 28, 28)).to(device)
model = NetworkQ5().to(device)
y = model(x)
print(y)

before flatten torch.Size([1, 32, 5, 5])
after torch.Size([1, 800])
tensor([[0.2313, 0.2009, 0.1854, 0.2302, 0.1523]], grad_fn=<SoftmaxBackward0>)


<ipython-input-26-c9432a451062>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.actout(self.output(x))   #  the same as p3


#### **5.2** The following training loop has 3 errors. Identify and fix them.

**This code snippet is not designed to be executed, it will not run**. The question is: what are the three conceptual errors in this training loop. Identify them in the code and add a comment next to their corresponding lines, and explain in the markdown block below why you think they are errors. You can assume that the following objects exist and you can use them: `data_loader`, `model`, `device`, `criterion`, and `optimizer`. Once you have identified them, fix them.

In [ ]:
for epoch in range(10):  

    running_loss = 0.0
    model.train() # e1: in the training loop we use model.train() instead of model.eval, so the parameters can be updated using backpropogation.
    for X, y in data_loader:
        # with torch.no_grad(): #e2: the same as e1, in the training loop, we want the gradient to update the parameters, so we cannot set as no_grad.
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad() # e3: the optimizer should be rest at the start of each batch, instead of in the end. Or we cannot update the parameters.
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

The specific answers are in the comment above.

<br>

---

<br>

## **Question 6** [15 points]

Given an RNN, with a single layer, that has been trained and is used to generate new outputs, like the one in the figure below:

<center><img src="https://drive.google.com/uc?id=1GmEyTghDKdkBx0gbFrw_KOn6zuKbNaHc" width="500"/></center> 

[link to the figure](https://drive.google.com/file/d/1GmEyTghDKdkBx0gbFrw_KOn6zuKbNaHc/view?usp=share_link) in case you cannot see it in the notebook.

<br>

If the dimension of my hidden vector is 20, and the dimension of my input and output is 35.

#### **Q6.1** What are the sizes of the matrices $W_{hh}$, $W_{xh}$ and $W_{hy}$? Use the next text block to write your answers:


$W_{hh}$:
- ... rows=20
- ... columns=20

$W_{xh}$:
- ... rows=20
- ... columns=35

$W_{hy}$:
- ... rows=35
- ... columns=20

#### **Q6.2** Given that this RNN network is trained and that we want to use it to generate new samples (words or characters), what operations do you perform on the outputs $y$ to generate a new sample in the sequence?

Input that output as the new input of RNN, repeat until I have generated the desired number of samples.

#### **Q6.3** Indicate which of the following statements is False, and explain why:

- a) LSTMs have two vectors that are passed: the cell state and the hidden state. The cell state is responsible for keeping 'longer term' memory in the system.

- b) The activations functions of the various gates in an LSTM are `tanh`, so that they can act as 'continuous switches' to pass or not pass information.

- c) LSTMs cells can be stacked to form deeper LSTMs architectures, in the same way that we can stack cells in RNNs.

B.  The activations functions of the various gates in an LSTM is sigmoid. Because its output is (0,1), it can act as the gate closed or open.

tanh in LSTM is just used to make gradient more stable in case of gradient vanishing or explosion. 

<br>

---

<br>

## **Question 7** [15 points]

Transformers are based on the self-attention mechanism. Answer the following questions about self-attention:

#### **Q7.1** The inputs of the network are embedded and positionally encoded before entering the first self-attention layer of the encoder. What are the three vectors that we create, and how are they used to produce the output of this self-attention layer? You can assume that we only have one head.

***\[ answer the question in your own words, but do not use more than ~100 words \]***

The three vectors are K, V, Q. Lets assume X=(x1, x2). We use WK,WQ,WV to get corresponding K=(k1, k2), V=(v1, v2), Q=(q1, q2). Take x1 as an example. scores=(q1*k_i)/sqrt(d_k), so for different x_i, we have different score, which correspond to their different importance to x1. Then softmax, and multiply them to their cooresponding v, and sum up, we can get the final z1. It consists  of all v_i and their corresponding importance to x1.

#### **Q7.2** Explain how the encoder and the decoder interact in the Transformer we saw in class.

***\[ answer the question in your own words, but do not use more than ~100 words \]***

In transformer, the encoder captures the meaning of the input sequence and passes this information to the decoder, which uses it to generate the output sequence.[4]

Let's assume the output of encoder zx. The decoder uses zx as the key and value input in the multi-attention block. The decoder takes the output of the masked multi-attention output as the query input of the latter multi-attention block. And then use multi-attention block to process it. In class, the transformer has 6 encoders and 6 decoders, we take the output of last encoder as zx, and in every decoder, especially in its multi-attention block, take zx as the key and value input of it.

<br>

---

<br>

## **Question 8** [15 points]

#### **Q8.1** Write custom dataset class that:

- Gets a sample from CelebA available from `torchvision`. Here is the link to the documentation: https://pytorch.org/vision/stable/generated/torchvision.datasets.CelebA.html

- Removes a squared block of the sample at a random location. *\[You want to generate images like the ones in figure-1 b) in this publication https://arxiv.org/pdf/1801.07939 (just given as a reference to what you are trying to achieve, no need to read anything from the paper here).\]*

- Returns the corrupted image as the input and the original image as the target.

Download the data with `torchvision`, and resize the images to have dimension 224 x 224.  Use one of the two templates$^1$ below as a starting point, and add any methods, code, or arguments you deem necessary. 

$^1$ your custom class can either inherit from the ``CelebA`` class or from the ``Dataset`` class, you can choose which one you prefer to use.



In [ ]:
### you can use this provided code in this cell or the one
### in the cell below (only use one of the two).
### Modify it as you see fit to produce what the question asks

class CorruptedCelebA(CelebA):
    def __init__(self, crop_size=80, **kwargs):
        super().__init__(**kwargs)
        self.crop_size = crop_size

    def __getitem__(self, idx):
        img, _ = super().__getitem__(idx)


        square = torch.zeros(3, self.crop_size, self.crop_size)
        convex = torch.randint(0,224-self.cropsize,1)
        img = img
        return corrupted_img, img

In [29]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  pass

Mounted at /content/drive


In [33]:
### you can use this provided code in this cell or the one
### in the cell above (only use one of the two). 
### Modify it as you see fit to produce what the question asks

class CorruptedCelebA(Dataset):
    def __init__(self, data, crop_size=80, **kwargs):
        super().__init__(**kwargs)
        
        self.crop_size = crop_size
        self.data = data

    def __getitem__(self, idx):
        img, _ = self.data[idx]
        corrupted_img = img


        h, w = corrupted_img.shape[1:]
        x = torch.randint(0, w - self.crop_size, (1,)).item()
        y = torch.randint(0, h - self.crop_size, (1,)).item()

        corrupted_img[:, y:y+self.crop_size, x:x+self.crop_size] = 0

        ### your code here
        return corrupted_img, img
    
    def __len__(self):
        return len(self.data)

In [ ]:
from torchvision.transforms import Compose, ToTensor 
import torchvision.datasets
transform=Compose([
            Resize((224, 224)),
            ToTensor(),
        ])
train_dataset = torchvision.datasets.CelebA(root="./", download=True, transform=transform, target_transform=None)

#### **Q8.2** Plot 32 of these samples and their corresponding targets.

In [ ]:
idxs = torch.randint(low=0, high=32, size=(6,))
fig, axs = plt.subplots(4, 8, figsize=(15, 5))
for i, idx in enumerate((idxs)):
    img, target = train_dataset[idx]
    axs[i].imshow(img, cmap="gray")
    axs[i].set_title(str(target))

#### **Q8.3** From the list below, choose two transforms that could help improve the generalisation of a model, and two that would damage its performance. Briefly discuss each of these transforms and how you think it would influence the model.

- `torchvision.transforms.CenterCrop()`
- `torchvision.transforms.Grayscale()`
- `torchvision.transforms.RandomAffine()`
- `torchvision.transforms.RandomHorizontalFlip()`
- `torchvision.transforms.RandomRotation()`
- `torchvision.transforms.RandomVerticalFlip()`
- `torchvision.transforms.GaussianBlur()`

improve: torchvision.transforms.RandomHorizontalFlip(), torchvision.transforms.RandomRotation()

bacasue it doesnot loss any information but do xome augmentaion

damege: torchvision.transforms.GaussianBlur(), torchvision.transforms.CenterCrop()

bacause its blur and loss imformation